In [0]:
# File location and type
file_location = "/Volumes/workspace/aerospace/flights/flights.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

1. Find out which airports when treated as Origin have the smallest and the largest Departure delays.

In [0]:
%sql
-- Average departure delay by origin airport
create or replace temporary view origin_depdelays as
select
  Origin,
  avg(DepDelay) as avg_dep_delay
from aerospace.flights
where DepDelay is not null
group by Origin;

-- Smallest average departure delay
select *
from origin_depdelays
order by avg_dep_delay asc
limit 10

In [0]:
%sql
-- Average departure delay by origin airport
create or replace temporary view origin_depdelays as
select
  Origin,
  avg(DepDelay) as avg_dep_delay
from aerospace.flights
where DepDelay is not null
group by Origin;

-- Largest average departure delay
select *
from origin_depdelays
order by avg_dep_delay desc
limit 10

2. Create a list of U.S. States with the number of airports that each of them has in descending order. Show this on a map of the U.S. (Hint: Use: a choropleth map. States are listed as USPS Abbreviations)

In [0]:
%sql
create or replace temporary view airports_per_state as
select
  STATE as state,
  count(*) as num_airports
from aerospace.airports
where STATE is not null
group by STATE
order by num_airports desc

In [0]:
%sql
select * from airports_per_state

Databricks visualization. Run in Databricks to view.

3. Create a list containing: DATE, NUM_OF_FLIGHTS, MAX_ARRIVAL_DELAY, AVG_ARRIVAL_DELAY. Please provide the date (from FlightDate) in the following format i.e. 2018-01-10. The maximum and Average delays should be provided in minutes but rounded to 2 digits after the decimal point.

In [0]:
%sql
create or replace temporary view daily_arrival_delays as
select
  date_format(FlightDate, 'yyyy-MM-dd') as DATE,
  count(*)                                 as NUM_OF_FLIGHTS,
  round(max(ArrDelay), 2)                  as MAX_ARRIVAL_DELAY,
  round(avg(ArrDelay), 2)                  as AVG_ARRIVAL_DELAY
from aerospace.flights
where ArrDelay is not null
group by date_format(FlightDate, 'yyyy-MM-dd')
order by DATE

In [0]:
%sql
select * from daily_arrival_delays

4. Show on a bar chart the number of flights per Carrier. Use the full name of the airline (from airlines.csv).

In [0]:
%sql
create or replace temporary view flights_per_carrier as
select
  f.Carrier,
  a.AIRLINE as AIRLINE_NAME,
  count(*)  as NUM_FLIGHTS
from aerospace.flights f
join aerospace.airlines a
  on f.Carrier = a.IATA_CODE
group by f.Carrier, a.AIRLINE
order by NUM_FLIGHTS desc

In [0]:
%sql
select * from flights_per_carrier

Databricks visualization. Run in Databricks to view.

5. Show on a pie chart the number of flights per Carrier but this time only show the individual results for the top 10 Carriers (measured by number of flights) and the rest show as one OTHER element. Hint: One of the solutions to this exercise may require the use of Temporary Views (Create or Replace Temporary View new_view as Select * from XXX). If it is done correctly OTHER should account for ~16% of all flights.

In [0]:
%sql
-- 1) Top 10 carriers by number of flights (by code)
create or replace temporary view top10_carriers as
select
  Carrier,
  count(*) as NUM_FLIGHTS
from aerospace.flights
group by Carrier
order by NUM_FLIGHTS desc
limit 10;

-- 2) Total flights per carrier 
create or replace temporary view flights_per_carrier_all as
select
  Carrier,
  count(*) as NUM_FLIGHTS
from aerospace.flights
group by Carrier;

-- 3) Top 10 by code + OTHER 
create or replace temporary view flights_top10_other as
select
  case
    when t.Carrier is not null then t.Carrier
    else 'OTHER'
  end as Carrier_Group,
  sum(a.NUM_FLIGHTS) as NUM_FLIGHTS
from flights_per_carrier_all a
left join top10_carriers t
  on a.Carrier = t.Carrier
group by
  case
    when t.Carrier is not null then t.Carrier
    else 'OTHER'
  end;

-- 4) Final dataset for the pie chart
select *
from flights_top10_other


Databricks visualization. Run in Databricks to view.